In [14]:
import pandas as pd
import mysql.connector
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import RepeatedKFold, KFold
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

%matplotlib inline

In [15]:
df = pd.read_csv(r"C:\Users\LENOVO\Downloads\Google-Playstore-Analysis-and-Prediction-System-main\Google-Playstore-Analysis-and-Prediction-System-main\installs_preprocessed.csv")

In [3]:
df.head

<bound method NDFrame.head of          Category  Rating_Count  Free  Size_in_Mb  Content_Rating  \
0               0           0.0     1        10.0               0   
1               1          64.0     1         2.9               0   
2               2           0.0     1         3.7               0   
3               3           5.0     1         1.8               0   
4               1           0.0     1         6.2               0   
...           ...           ...   ...         ...             ...   
2237008        43       16775.0     1        77.0               1   
2237009        20           0.0     1        44.0               0   
2237010        20           0.0     1        29.0               0   
2237011        23           8.0     1        10.0               0   
2237012        35          12.0     1         5.2               0   

         Ad_Supported  In_App_Purchases  Editors_Choice  Installs_Category  
0                   0                 0               0         

In [3]:
# !pip install pymssql pandas imbalanced-learn scikit-learn

In [18]:

# Connect to SQL Server
conn = mysql.connector.connect(host="localhost", user="root", password="root", database="GooglePlayStore")

# Fetch Data
query = "SELECT * FROM installs"
df = pd.read_sql(query, conn)

# Close connection
conn.close()

df = df.drop('id', axis=1)
# Show first few rows
print(df.head())



C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3444\4068824899.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


  Category  Free  Size_in_Mb Content_Rating  Ad_Supported  In_App_Purchases  \
0     39.0     1        94.0            1.0             1                 0   
1     20.0     1        12.0            0.0             0                 0   
2     19.0     1         3.9            0.0             0                 0   
3     26.0     1         2.7            0.0             1                 1   
4     30.0     1        25.0            0.0             0                 0   

  Installs_Category  Rating_Count  Editors_Choice  
0               0.0             0               0  
1               0.0            16               0  
2               0.0             0               0  
3               0.0            21               0  
4               0.0             8               0  


In [16]:
df.Installs_Category.value_counts()

Installs_Category
0    1745458
2     247609
4     104284
1      71936
6      30545
3      25065
7       5789
5       5270
9        629
8        428
Name: count, dtype: int64

## Feature Extraction

In [4]:
df.shape

(2237013, 9)

In [17]:
import pandas as pd

# Assuming df is your DataFrame

# Filter out rows where Transformed_Rating is 0
zero_rating_rows = df[df["Installs_Category"] == 0]

# Randomly sample 500,000 rows from the filtered DataFrame
rows_to_drop = zero_rating_rows.sample(n=1600000, random_state=42)

# Drop these rows from the original DataFrame
df = df.drop(rows_to_drop.index).reset_index(drop=True)

# Check the new shape
print("New DataFrame Shape:", df.shape)

New DataFrame Shape: (637013, 9)


In [18]:
df.Installs_Category.value_counts()

Installs_Category
2    247609
0    145458
4    104284
1     71936
6     30545
3     25065
7      5789
5      5270
9       629
8       428
Name: count, dtype: int64

In [19]:
from imblearn.combine import SMOTETomek
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE

# Load dataset
X = df.drop("Installs_Category", axis=1)  # Features
Y = df["Installs_Category"]  # Target variable

# Original class distribution
print("🔹 Original Class Distribution:", Counter(Y))
# Check the new shape
print("New DataFrame Shape:", df.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.3, random_state=7,stratify=Y)

# # Apply SMOTETomek (Combination)
smote_tomek = SMOTETomek(random_state=7)
X_train_resampled, Y_train_resampled = smote_tomek.fit_resample(X_train, Y_train)
print("🔹 After SMOTETomek:", Counter(Y_train_resampled))

# # Choose the final resampled data (you can pick any of the resampled data)
# X = X_combined  
# Y = y_combined  

# # Ensure X and y have the same number of samples after resampling
print(f"X shape: {X_train_resampled.shape}, Y shape: {Y_train_resampled.shape}")

🔹 Original Class Distribution: Counter({2: 247609, 0: 145458, 4: 104284, 1: 71936, 6: 30545, 3: 25065, 7: 5789, 5: 5270, 9: 629, 8: 428})
New DataFrame Shape: (637013, 9)
🔹 After SMOTETomek: Counter({0: 168099, 7: 167319, 3: 167045, 6: 166109, 5: 164084, 1: 163560, 4: 162764, 8: 161244, 9: 159404, 2: 158222})
X shape: (1637850, 8), Y shape: (1637850,)


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
import pandas as pd

# Split features and target variable
X = df.drop(['Installs_Category'], axis=1)
y = df['Installs_Category']

# Train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=7, stratify=y)

# Initialize DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(random_state=7)

# Fit the model
dt_classifier.fit(X_train, Y_train)

# Predict and evaluate
y_pred = dt_classifier.predict(X_test)

# Extract feature importances
importances = dt_classifier.feature_importances_

# Rank the features
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Display the ranked features
mean_imp = dt_classifier.feature_importances_.mean()
selected_features = X_train.columns[dt_classifier.feature_importances_ > mean_imp]
print("Selected Features Based on Decision Tree Importance:")
print(selected_features)

# Perform Recursive Feature Elimination (RFE)
rfe = RFE(DecisionTreeClassifier(random_state=7))
rfe.fit(X_train, Y_train)

# Print feature rankings
print("\nRFE Feature Rankings:")
for rank, feature in sorted(zip(rfe.ranking_, X_train.columns)):
    print(f"{feature}: {rank}")


Selected Features Based on Decision Tree Importance:
Index(['Rating_Count', 'Size_in_Mb'], dtype='object')

RFE Feature Rankings:
Category: 1
Content_Rating: 1
Rating_Count: 1
Size_in_Mb: 1
In_App_Purchases: 2
Ad_Supported: 3
Free: 4
Editors_Choice: 5


Models


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score

# Target variable
y = df['Installs_Category']

# Feature Selection (Removing target and leakage features)
X = df.drop(['Installs_Category'], axis=1)

# Split dataset into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=7)

# Label Encoding for Target Variable
label_encoder = LabelEncoder()
Y_train_encoded = label_encoder.fit_transform(Y_train)
Y_test_encoded = label_encoder.transform(Y_test)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

results = []  # Store results for comparison

def evaluate_model(model, X_test, Y_test_encoded):
    Y_pred_encoded = model.predict(X_test)
    accuracy = accuracy_score(Y_test_encoded, Y_pred_encoded)
    f1 = f1_score(Y_test_encoded, Y_pred_encoded, average='weighted')

    print(f"Model: {model.__class__.__name__}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("-" * 40)

    # Store results
    results.append({"Model": model.__class__.__name__, "Accuracy": accuracy, "F1 Score": f1})


In [13]:
# # Applying Model CatBoost Model
import pickle
Cat_Boost = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=8, verbose=200, random_seed=7)
# Cat_Boost = CatBoostClassifier(verbose = 0)
Cat_Boost.fit(X_train_resampled, Y_train_resampled)
cb_ac=Cat_Boost.score(X_train, Y_train)
print("CatBoost_Accuracy: ",cb_ac)

with open("rating_model_one.pkl", "wb") as f:
    pickle.dump(Cat_Boost, f)

0:	learn: 2.0324097	total: 1.16s	remaining: 1m 54s
99:	learn: 1.0346493	total: 1m 41s	remaining: 0us
CatBoost_Accuracy:  0.6554469877439626


In [10]:
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, roc_curve, auc
import matplotlib.pyplot as plt
import pickle

# Train CatBoost model
Cat_Boost = CatBoostClassifier(verbose=0)
Cat_Boost.fit(X_train, Y_train)

# Model Accuracy
cb_ac = Cat_Boost.score(X_train, Y_train)
print("CatBoost Accuracy: ", cb_ac)

# Predictions
y_pred = Cat_Boost.predict(X_test)

# **F1-Score Calculation**
f1 = f1_score(Y_test, y_pred, average="weighted")  # Use 'weighted' for multi-class
print("F1 Score:", f1)

# **ROC Curve Calculation**
y_prob = Cat_Boost.predict_proba(X_test)  # Get probability estimates
if len(set(Y_test)) == 2:  # Binary classification
    fpr, tpr, _ = roc_curve(Y_test, y_prob[:, 1])  # Only second column for positive class
    roc_auc = auc(fpr, tpr)

    # Plot ROC Curve
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color="blue", lw=2, label=f"ROC curve (area = {roc_auc:.2f})")
    plt.plot([0, 1], [0, 1], color="gray", linestyle="--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve - CatBoost")
    plt.legend(loc="lower right")
    plt.show()
    print("ROC AUC Score:", roc_auc)
else:
    print("ROC curve is not applicable for multi-class classification.")

# Save the model
with open("rating_model.pkl", "wb") as f:
    pickle.dump(Cat_Boost, f)


0:	learn: 2.0418925	total: 4.66s	remaining: 7m 41s
99:	learn: 1.0545019	total: 4m 33s	remaining: 0us


NameError: name 'evaluate_model' is not defined

In [13]:
# Applying Model CatBoost Model
import pickle
Cat_Boost = CatBoostClassifier(verbose = 200, n_estimators = 2500, learning_rate=0.1, depth=8,early_stopping_rounds=300)
# Cat_Boost = CatBoostClassifier(verbose = 0)
Cat_Boost.fit(X_train_scaled, Y_train_encoded)
cb_ac=Cat_Boost.score(X_train_scaled, Y_train_encoded)
print("CatBoost_Accuracy: ",cb_ac)

with open("installs_model.pkl", "wb") as f:
    pickle.dump(Cat_Boost, f)

0:	learn: 2.0275818	total: 536ms	remaining: 22m 20s
200:	learn: 1.3014477	total: 2m 34s	remaining: 29m 32s
400:	learn: 1.2867157	total: 5m 58s	remaining: 31m 16s
600:	learn: 1.2773017	total: 19m 40s	remaining: 1h 2m 10s


KeyboardInterrupt: 

In [15]:
#Applying Model Logistic Regression


log_reg = LogisticRegression(max_iter=500, class_weight='balanced')
log_reg.fit(X_train_scaled, Y_train_encoded)
evaluate_model(log_reg, X_test_scaled, Y_test_encoded)


Model: LogisticRegression
Accuracy: 0.3590
F1 Score: 0.3975
----------------------------------------


In [ ]:
#Applying Model RandomForest

rf_clf = RandomForestClassifier(n_estimators=100, random_state=7, class_weight='balanced')
rf_clf.fit(X_train_scaled, Y_train_encoded)
evaluate_model(rf_clf, X_test_scaled, Y_test_encoded)


Model: RandomForestClassifier
Accuracy: 0.4188
F1 Score: 0.5202
----------------------------------------


In [ ]:
#Applying Model DecisionTree Classifier

dt_clf = DecisionTreeClassifier(random_state=7)
dt_clf.fit(X_train_scaled, Y_train_encoded)
evaluate_model(dt_clf, X_test_scaled, Y_test_encoded)


Model: DecisionTreeClassifier
Accuracy: 0.7723
F1 Score: 0.6926
----------------------------------------


In [ ]:
#Applying Model Naive Bayesian

nb_clf = GaussianNB()
nb_clf.fit(X_train_scaled, Y_train_encoded)
evaluate_model(nb_clf, X_test_scaled, Y_test_encoded)

Model: GaussianNB
Accuracy: 0.6015
F1 Score: 0.6205
----------------------------------------


In [ ]:
# Applying K-Nearest Neighbors


knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train_scaled, Y_train_encoded)
evaluate_model(knn_clf, X_test_scaled, Y_test_encoded)


Model: KNeighborsClassifier
Accuracy: 0.7618
F1 Score: 0.6932
----------------------------------------


In [ ]:
# Fine-tuning Linear Discriminant Analysis (LDA)
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(X_train_scaled, Y_train_encoded)
evaluate_model(lda_clf, X_test_scaled, Y_test_encoded)


Model: LinearDiscriminantAnalysis
Accuracy: 0.7712
F1 Score: 0.6867
----------------------------------------


In [ ]:
# Applying Gradient Boosting Classifier


gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=7)
gb_clf.fit(X_train_scaled, Y_train_encoded)
evaluate_model(gb_clf, X_test_scaled, Y_test_encoded)


Model: GradientBoostingClassifier
Accuracy: 0.7801
F1 Score: 0.6861
----------------------------------------


In [ ]:
# Applying AdaBoost Classifier

ada_clf = AdaBoostClassifier(n_estimators=100, learning_rate=0.1, random_state=7)
ada_clf.fit(X_train_scaled, Y_train_encoded)
evaluate_model(ada_clf, X_test_scaled, Y_test_encoded)


Model: AdaBoostClassifier
Accuracy: 0.7798
F1 Score: 0.6834
----------------------------------------


In [ ]:
# Applying XGBoost Classifier


xgb_clf = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=7)
xgb_clf.fit(X_train_scaled, Y_train_encoded)
evaluate_model(xgb_clf, X_test_scaled, Y_test_encoded)


Model: XGBClassifier
Accuracy: 0.7804
F1 Score: 0.6869
----------------------------------------
